In [1]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
from langchain.tools.gmail.utils import build_resource_service, get_gmail_credentials
from langchain.agents.agent_toolkits import GmailToolkit
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [3]:
!export PROJECT_ID="gcp-mira-demo"
project = "gcp-mira-demo"
os.environ["PROJECT_ID"] = project

In [4]:
from config import (OPENAI_API_KEY)

INFO: [src/config.py:49 - <module>()] Namespace File not found, setting job namespace as default
INFO: [src/config.py:84 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [src/config.py:84 - get_environ_flag()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [src/config.py:84 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [src/config.py:84 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [src/config.py:196 - <module>()] LLM types loaded ['OpenAI-GPT3.5', 'OpenAI-GPT4', 'Cohere', 'VertexAI-Text', 'VertexAI-Chat', 'VertexAI-Chat', 'VertexAI-ModelGarden-LLAMA2-Chat']


In [5]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [6]:
email_template = """
    You are working for {state} State Medicaid Agency. Create only the email message body for recipient: {recipient} \n\n
    \n\n Use text delimited by triple backticks to create the email body text:'''{email_body}'''
"""

In [7]:
email_body_prompt = ChatPromptTemplate.from_template(email_template)
email_body_prompt

ChatPromptTemplate(input_variables=['email_body', 'recipient', 'state'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['email_body', 'recipient', 'state'], template="\n    You are working for {state} State Medicaid Agency. Create only the email message body for recipient: {recipient} \n\n\n    \n\n Use text delimited by triple backticks to create the email body text:'''{email_body}'''\n"))])

In [8]:
llm_chain = LLMChain(prompt=email_body_prompt, llm=llm)

In [9]:
prompt_string = """Create an email to this medicaid applicant that is missing income verification asking them to email a pay stub from their employers"""

In [10]:
sender='mira-admin@lavjain.altostrat.com'

In [11]:
recipient='hello@google.com'

In [12]:
state='NYS'

In [13]:
message = llm_chain.run(state=state,recipient=recipient,email_body=prompt_string)
print(f"{message}")


Dear Applicant, 

We are writing to you regarding your application for New York State Medicaid. We have noticed that we are missing income verification from your application. 

In order to complete your application, we need you to email a pay stub from your employer to us at evekhm@google.com. 

We appreciate your cooperation in this matter and thank you for your time. 

Sincerely, 
NYS State Medicaid Agency


In [14]:
subject_template = """
    Create a subject for the following email: {email_body} \n\n 
"""

In [15]:
subject_prompt = ChatPromptTemplate.from_template(subject_template)

In [16]:
llm_chain = LLMChain(prompt=subject_prompt, llm=llm)

In [17]:
subject = llm_chain.run(email_body=message)
print(f"{subject}")

Subject: Income Verification Request for New York State Medicaid Application


For the steps below, you will need to locally generate token and credentials:
```shell
cd components/llm_service/src
gcloud secrets versions access latest --secret=tools-gmail-oauth-token > data/token.json
gcloud secrets versions access latest --secret=tools-gmail-client-secrets > data/credentials.json
```


In [18]:
credentials = get_gmail_credentials(
    token_file="./data/token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="./data/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [19]:
tools = toolkit.get_tools()

In [20]:
input = {'to':recipient, 'subject':subject,'message':message}

In [21]:
tools[1].invoke(input=input)

'Message sent. Message Id: 18bf302f7174cd27'